In [1]:
#install
!pip install ortools
!pip install openrouteservice
!pip install folium
!pip install geopy

#library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import folium
import openrouteservice
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from openrouteservice import distance_matrix
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from ortools.sat.python import cp_model

## New Distance Matrix

In [2]:
#Load updated matrix

# URLs of the two CSV files on GitHub
url1 = 'https://raw.githubusercontent.com/linneverh/logistics/refs/heads/main/Data/final_driving_distance_matrix_km.csv'
url2 = 'https://raw.githubusercontent.com/linneverh/logistics/refs/heads/main/Data/final_state_stats_with_capacity.csv'

In [3]:
# Dataframe with corresponding capacities
capacities_31012025 = pd.read_csv(url2)
print(capacities_31012025)

    Unnamed: 0       Federal Entity  Order Count   Latitude   Longitude  \
0            0                Depot            0  19.743368  -99.128924   
1            1       Aguascalientes          841  21.882300 -102.282600   
2            2      Baja California          787  32.522499 -117.046623   
3            3  Baja California Sur          885  23.062283 -109.699951   
4            4             Campeche          307  19.850000  -90.530000   
5            5              Chiapas          661  16.759700  -93.113600   
6            6            Chihuahua          892  28.635300 -106.088900   
7            7             Coahuila         1257  25.438900 -100.973300   
8            8               Colima          732  19.233000 -103.717000   
9            9              Durango          257  24.027700 -104.653200   
10          10           Guanajuato         3548  21.116667 -101.683334   
11          11             Guerrero          665  16.848824  -99.912437   
12          12           

In [4]:
capacities_filtered = capacities_31012025[['Federal Entity', 'Total Capacity (Weight)']]

# Display the filtered dataframe
print("Filtered capacities dataframe:")
capacities_filtered

Filtered capacities dataframe:


,Federal Entity,Total Capacity (Weight)
0,Depot,NaN
1,Aguascalientes,NaN
2,Baja California,332.708
3,Baja California Sur,383.758
4,Campeche,NaN
5,Chiapas,81.140
6,Chihuahua,13.457
7,Coahuila,128.478
8,Colima,135.388
9,Durango,14.042


# Driving distance

In [5]:
# Driving distance maxtrix
driving_distance = pd.read_csv(url1)
driving_distance

,Unnamed: 0,Depot,Aguascalientes,Baja California,Baja California Sur,Campeche,Chiapas,Chihuahua,Coahuila,Colima,...,Quintana Roo,San Luis Potosí,Sinaloa,Sonora,Tabasco,Tamaulipas,Tlaxcala,Veracruz,Yucatán,Zacatecas
0,Depot,0.00,452.71,2734.72,1820.16,1165.11,859.49,1403.67,800.89,684.13,...,1638.40,371.27,1193.69,1861.58,779.67,435.11,141.64,293.56,1338.50,574.46
1,Aguascalientes,453.11,0.00,2502.58,1532.34,1613.98,1308.37,947.42,483.93,410.55,...,2087.28,164.40,905.88,1573.76,1228.54,616.13,590.51,742.93,1787.38,118.21
2,Baja California,2919.89,2341.00,0.00,1652.82,4080.26,3774.65,1385.84,2123.18,2574.80,...,4553.56,2412.63,1750.84,1056.45,3694.82,2953.60,3056.79,3209.21,4253.65,2226.71
3,Baja California Sur,1816.92,1529.97,1656.14,0.00,2977.29,2671.67,1288.99,1591.12,1471.83,...,3450.58,1637.94,647.86,918.95,2591.85,2090.02,1953.82,2106.23,3150.68,1373.32
4,Campeche,1184.57,1623.39,3904.89,2990.33,0.00,624.80,2574.35,1864.64,1854.30,...,473.26,1541.95,2363.86,3031.75,385.84,1277.77,1051.76,948.22,173.36,1745.15
5,Chiapas,879.58,1318.40,3599.90,2685.35,624.54,0.00,2269.37,1559.66,1549.32,...,1097.84,1236.97,2058.88,2726.76,239.11,972.78,746.77,643.24,797.94,1440.16
6,Chihuahua,1407.10,950.28,1552.48,1287.90,2567.97,2262.36,0.00,732.46,1364.54,...,3041.27,1021.91,736.56,688.81,2182.54,1316.06,1544.50,1696.92,2741.37,835.98
7,Coahuila,800.04,482.53,2281.16,1585.73,1871.27,1565.66,726.00,0.00,896.80,...,2344.57,446.74,959.27,1627.15,1485.83,588.77,937.44,1089.85,2044.66,376.64
8,Colima,685.82,412.00,2394.35,1479.79,1846.18,1540.57,1362.96,899.47,0.00,...,2319.48,519.97,853.33,1521.21,1460.75,972.05,822.72,975.13,2019.58,533.76
9,Durango,859.65,402.82,1994.94,1080.38,2020.51,1714.90,691.08,514.12,817.08,...,2493.81,474.45,453.91,1121.79,1635.08,901.25,997.05,1149.46,2193.91,288.52


In [6]:
# Set state names as index and column names for the driving distance matrix
driving_distance = driving_distance.set_index('Unnamed: 0')
driving_distance.index.name = None
driving_distance.columns = driving_distance.index  # ensure square matrix

In [7]:
# Extract state order from the distance matrix
ordered_states = driving_distance.index.tolist()

# Reorder capacity dataframe using state names
capacities_filtered = capacities_filtered.set_index('Federal Entity').reindex(ordered_states).reset_index()

# Rename capacity column for consistency
capacities_filtered = capacities_filtered.rename(columns={'Total Capacity (Weight)': 'Capacity Value Weight'})

# Fill missing capacities with 0 and make sure depot (row 0) has demand = 0
capacities_filtered['Capacity Value Weight'] = capacities_filtered['Capacity Value Weight'].fillna(0)
capacities_filtered.loc[capacities_filtered.index == 0, 'Capacity Value Weight'] = 0

# Create demand vector
demand = capacities_filtered['Capacity Value Weight'].round().astype(int).tolist()

# Filter states to keep only those with demand > 0 or depot (index 0)
states_to_keep = [state for state, d in zip(ordered_states, demand) if d > 0 or state == ordered_states[0]]

# Rebuild distance matrix and demand vector
states_index = pd.Index(states_to_keep)

filtered_distance = driving_distance.reindex(index=states_index, columns=states_index, fill_value=0)

demand_map = dict(zip(ordered_states, demand))
filtered_demand = [demand_map[state] for state in filtered_distance.index]

# Final check
print("\nFiltered distance matrix:")
display(filtered_distance)
print("\nFiltered demand vector:")
print(filtered_demand)


Filtered distance matrix:


,Depot,Baja California,Baja California Sur,Chiapas,Chihuahua,Coahuila,Colima,Durango,Guanajuato,Guerrero,...,Quintana Roo,San Luis Potosí,Sinaloa,Sonora,Tabasco,Tamaulipas,Tlaxcala,Veracruz,Yucatán,Zacatecas
Depot,0.00,2734.72,1820.16,859.49,1403.67,800.89,684.13,856.91,335.06,419.73,...,1638.40,371.27,1193.69,1861.58,779.67,435.11,141.64,293.56,1338.50,574.46
Baja California,2919.89,0.00,1652.82,3774.65,1385.84,2123.18,2574.80,2183.47,2633.65,3167.82,...,4553.56,2412.63,1750.84,1056.45,3694.82,2953.60,3056.79,3209.21,4253.65,2226.71
Baja California Sur,1816.92,1656.14,0.00,2671.67,1288.99,1591.12,1471.83,1080.50,1530.68,2064.84,...,3450.58,1637.94,647.86,918.95,2591.85,2090.02,1953.82,2106.23,3150.68,1373.32
Chiapas,879.58,3599.90,2685.35,0.00,2269.37,1559.66,1549.32,1722.61,1200.76,1107.81,...,1097.84,1236.97,2058.88,2726.76,239.11,972.78,746.77,643.24,797.94,1440.16
Chihuahua,1407.10,1552.48,1287.90,2262.36,0.00,732.46,1364.54,692.48,1080.18,1815.05,...,3041.27,1021.91,736.56,688.81,2182.54,1316.06,1544.50,1696.92,2741.37,835.98
Coahuila,800.04,2281.16,1585.73,1565.66,726.00,0.00,896.80,510.10,612.43,1207.99,...,2344.57,446.74,959.27,1627.15,1485.83,588.77,937.44,1089.85,2044.66,376.64
Colima,685.82,2394.35,1479.79,1540.57,1362.96,899.47,0.00,816.20,412.70,648.91,...,2319.48,519.97,853.33,1521.21,1460.75,972.05,822.72,975.13,2019.58,533.76
Durango,859.65,1994.94,1080.38,1714.90,691.08,514.12,817.08,0.00,532.72,1267.59,...,2493.81,474.45,453.91,1121.79,1635.08,901.25,997.05,1149.46,2193.91,288.52
Guanajuato,337.78,2447.28,1532.73,1193.03,1075.27,611.78,410.93,528.51,0.00,745.73,...,1971.94,192.48,906.26,1574.14,1113.21,609.55,475.18,627.59,1672.04,246.07
Guerrero,431.10,2984.80,2070.25,1096.45,1822.90,1220.12,649.34,1276.14,754.29,0.00,...,1875.36,790.50,1443.78,2111.66,1016.62,821.83,418.14,561.49,1575.45,993.70



Filtered demand vector:
[0, 333, 384, 81, 13, 128, 135, 14, 944, 215, 483, 3912, 164, 648, 5598, 138, 2919, 160, 246, 132, 1286, 514, 248, 205, 4, 308, 94, 379, 419, 64]


In [8]:
print("States in use:", list(filtered_distance.index))
print("Total demand (kg):", sum(filtered_demand))

States in use: ['Depot', 'Baja California', 'Baja California Sur', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán', 'Morelos', 'México (state)', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas']
Total demand (kg): 20168


In [9]:
print("Depot:", filtered_distance.index[0])
print("Delivery States:", list(filtered_distance.index[1:]))

Depot: Depot
Delivery States: ['Baja California', 'Baja California Sur', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán', 'Morelos', 'México (state)', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas']


## First Capacitated Facility Location Problem

In [10]:
max_distance = filtered_distance.values.max()
print(f"Max one-way trip distance: {max_distance:.2f} km")

Max one-way trip distance: 4553.56 km


In [34]:
def create_data_model():
    """Stores the data for the problem with mixed vehicle types."""
    max_dists = [4000] * 0 + [6000] * 4

    data = {}
    data['distance_matrix'] = filtered_distance.values.tolist()
    data['demands'] = filtered_demand
    data['vehicle_max_distances'] = max_dists
    data['vehicle_capacities'] = [16000] * len(max_dists)
    data['num_vehicles'] = len(max_dists)
    data['depot'] = 0
    return data

In [12]:
def print_solution(data, manager, routing, solution, labels):
    """Prints routes using state names and returns total distance."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        capacity_used = 0
        plan_output = f'\n🚚 Route for vehicle {vehicle_id} (Depot: {labels[0]}):\n'

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand = data['demands'][node_index]
            capacity_used += demand
            plan_output += f' → {labels[node_index]} (Demand: {demand}, Used: {capacity_used})\n'
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        plan_output += f' → Return to {labels[0]} (Depot)\n'
        plan_output += f'Distance: {route_distance} km\n'
        print(plan_output)
        total_distance += route_distance

    print(f'\n📦 Total distance: {total_distance:.2f} km')
    return total_distance

In [15]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def main():
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'],
                                           data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Define distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Define demand callback
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    # Add capacity constraint
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # no slack
        data['vehicle_capacities'],
        True,  # start cumul to zero
        'Capacity'
    )

    routing.AddDimension(
        transit_callback_index,
        0,
        10000,  # very high upper bound to allow customization per vehicle
        True,
        'Distance'
    )
    distance_dimension = routing.GetDimensionOrDie('Distance')

    # Set max travel distances per vehicle
    for vehicle_id, max_dist in enumerate(data['vehicle_max_distances']):
        distance_dimension.CumulVar(routing.End(vehicle_id)).SetMax(max_dist)

    # Optional cost weighting for span
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Solver parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.time_limit.seconds = 10

    # Solve
    solution = routing.SolveWithParameters(search_parameters)

    state_labels = states_to_keep

    if solution:
        return print_solution(data, manager, routing, solution, state_labels)
    else:
        print("No solution found.")
        return None


In [35]:
baseline_total_distance = main()


🚚 Route for vehicle 0 (Depot: Depot):
 → Depot (Demand: 0, Used: 0)
 → Sonora (Demand: 205, Used: 205)
 → Baja California (Demand: 333, Used: 538)
 → Chihuahua (Demand: 13, Used: 551)
 → San Luis Potosí (Demand: 514, Used: 1065)
 → Return to Depot (Depot)
Distance: 5507 km


🚚 Route for vehicle 1 (Depot: Depot):
 → Depot (Demand: 0, Used: 0)
 → Hidalgo (Demand: 483, Used: 483)
 → Tamaulipas (Demand: 308, Used: 791)
 → Nuevo León (Demand: 2919, Used: 3710)
 → Coahuila (Demand: 128, Used: 3838)
 → Zacatecas (Demand: 64, Used: 3902)
 → Durango (Demand: 14, Used: 3916)
 → Sinaloa (Demand: 248, Used: 4164)
 → Baja California Sur (Demand: 384, Used: 4548)
 → Nayarit (Demand: 138, Used: 4686)
 → Colima (Demand: 135, Used: 4821)
 → Jalisco (Demand: 3912, Used: 8733)
 → Guanajuato (Demand: 944, Used: 9677)
 → Michoacán (Demand: 164, Used: 9841)
 → Querétaro (Demand: 132, Used: 9973)
 → Return to Depot (Depot)
Distance: 5439 km


🚚 Route for vehicle 2 (Depot: Depot):
 → Depot (Demand: 0, Used: 

## FLP

In [36]:
def run_flp(distance_matrix, demand_vector, cluster_ids, depot_index=0, max_warehouses=2):
    num_nodes = len(distance_matrix)
    EF = 1  # scaling factor; keep 1 if just minimizing distance

    model = cp_model.CpModel()

    # Binary: is warehouse opened at node i?
    open_vars = [model.NewBoolVar(f"open[{i}]") for i in range(num_nodes)]

    # Binary: is node j served by warehouse at i?
    assign = [[model.NewBoolVar(f"assign[{i}][{j}]") for j in range(num_nodes)] for i in range(num_nodes)]

    # Each node must be assigned to one warehouse
    for j in range(num_nodes):
        model.Add(sum(assign[i][j] for i in range(num_nodes)) == 1)

    # Can only assign to open warehouses
    for i in range(num_nodes):
        for j in range(num_nodes):
            model.Add(assign[i][j] <= open_vars[i])

    # Fix depot (cluster 0) to be open
    model.Add(open_vars[depot_index] == 1)

    # Open exactly max_warehouses (1 depot + 1 new)
    model.Add(sum(open_vars) == max_warehouses)

    # Objective: minimize total emissions (distance × demand)
    total_cost_terms = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            cost = int(distance_matrix[i][j] * demand_vector[j] * EF)
            total_cost_terms.append(assign[i][j] * cost)

    model.Minimize(sum(total_cost_terms))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        selected_warehouses = [i for i in range(num_nodes) if solver.Value(open_vars[i])]
        assignments = [max(range(num_nodes), key=lambda i: solver.Value(assign[i][j])) for j in range(num_nodes)]

        print("✅ Warehouses selected at states:", [cluster_ids[i] for i in selected_warehouses])
        return selected_warehouses, assignments
    else:
        print("❌ No solution found.")
        return None, None


In [37]:
selected_warehouses, assignments = run_flp(
    filtered_distance.values.tolist(),
    filtered_demand,
    cluster_ids=states_to_keep,  # rename for clarity
    depot_index=0,
    max_warehouses=2      # Total 2 warehouses (1 existing + 1 new)
)

✅ Warehouses selected at states: ['Depot', 'Jalisco']


In [38]:
from collections import defaultdict

def group_assignments_by_warehouse(assignments, selected_warehouses):
    clusters_by_wh = defaultdict(list)
    for node_idx, wh_idx in enumerate(assignments):
        clusters_by_wh[wh_idx].append(node_idx)
    return clusters_by_wh

# Returns: dict { warehouse_index -> list of node indices assigned }
assignments_by_wh = group_assignments_by_warehouse(assignments, selected_warehouses)

## Re-run CVRP

In [40]:
def run_cvrp_for_warehouse(wh_index, assigned_node_indices, full_distance_df, full_demand_vector, cluster_ids):
    # Step 1: remap so warehouse is first
    nodes = [wh_index] + [i for i in assigned_node_indices if i != wh_index]

    # Step 2: subset distance and demand
    sub_matrix = full_distance_df.iloc[nodes, :].iloc[:, nodes].values.tolist()
    sub_demand = [full_demand_vector[i] for i in nodes]
    cluster_labels = [cluster_ids[i] for i in nodes]  # real cluster numbers for printing

    def create_data_model():
        return {
            'distance_matrix': sub_matrix,
            'demands': sub_demand,
            'vehicle_capacities': [16000] * 4,
            'num_vehicles': 4,
            'depot': 0
        }

    from ortools.constraint_solver import pywrapcp, routing_enums_pb2

    def print_solution(data, manager, routing, solution, labels):
        total_distance = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            route_distance = 0
            capacity_used = 0
            plan_output = f'\n🚚 Route for vehicle {vehicle_id} (Warehouse: {cluster_labels[0]}):\n'

            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                demand = data['demands'][node_index]
                capacity_used += demand
                plan_output += f' → {cluster_labels[node_index]} (Demand: {demand}, Used: {capacity_used})\n'
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

            plan_output += f' → Return to {cluster_labels[0]} (Depot)\n'
            plan_output += f'Distance: {route_distance} km\n'
            print(plan_output)
            total_distance += route_distance

        return total_distance

    def solve_cvrp():
        data = create_data_model()
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'],
                                               data['depot'])
        routing = pywrapcp.RoutingModel(manager)

        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return int(data['distance_matrix'][from_node][to_node])
        transit_cb = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

        # Add distance dimension
        routing.AddDimension(
            transit_cb,
            0,        # no slack
            6000,     # vehicle max travel distance (km)
            True,     # start cumul at zero
            'Distance'
        )
        distance_dimension = routing.GetDimensionOrDie('Distance')
        distance_dimension.SetGlobalSpanCostCoefficient(100)


        def demand_callback(from_index):
            return data['demands'][manager.IndexToNode(from_index)]
        demand_cb = routing.RegisterUnaryTransitCallback(demand_callback)

        routing.AddDimensionWithVehicleCapacity(
            demand_cb, 0, data['vehicle_capacities'], True, 'Capacity'
        )

        search_params = pywrapcp.DefaultRoutingSearchParameters()
        search_params.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        search_params.time_limit.seconds = 10

        solution = routing.SolveWithParameters(search_params)
        return print_solution(data, manager, routing, solution, cluster_labels) if solution else None

    total_distance = solve_cvrp()
    print(f"\n✅ Finished CVRP for warehouse: {cluster_labels[0]}. Total distance: {total_distance:.2f} km")
    return total_distance

In [41]:
total_distance_with_2_warehouses = 0

for wh_idx, assigned_nodes in assignments_by_wh.items():
    print(f"\n🚚 Running CVRP for warehouse index {wh_idx} ({states_to_keep[wh_idx]})")

    distance = run_cvrp_for_warehouse(
        wh_index=wh_idx,
        assigned_node_indices=assigned_nodes,
        full_distance_df=filtered_distance,
        full_demand_vector=filtered_demand,
        cluster_ids=states_to_keep  # renamed for clarity
    )

    total_distance_with_2_warehouses += distance

print(f"\n📉 Total distance with 2 warehouses: {total_distance_with_2_warehouses:.2f} km")


🚚 Running CVRP for warehouse index 0 (Depot)

🚚 Route for vehicle 0 (Warehouse: Depot):
 → Depot (Demand: 0, Used: 0)
 → Hidalgo (Demand: 483, Used: 483)
 → Tamaulipas (Demand: 308, Used: 791)
 → Veracruz (Demand: 379, Used: 1170)
 → Chiapas (Demand: 81, Used: 1251)
 → Oaxaca (Demand: 160, Used: 1411)
 → Puebla (Demand: 246, Used: 1657)
 → Tlaxcala (Demand: 94, Used: 1751)
 → Guerrero (Demand: 215, Used: 1966)
 → Morelos (Demand: 648, Used: 2614)
 → México (state) (Demand: 5598, Used: 8212)
 → Return to Depot (Depot)
Distance: 3232 km


🚚 Route for vehicle 1 (Warehouse: Depot):
 → Depot (Demand: 0, Used: 0)
 → Return to Depot (Depot)
Distance: 0 km


🚚 Route for vehicle 2 (Warehouse: Depot):
 → Depot (Demand: 0, Used: 0)
 → Tabasco (Demand: 4, Used: 4)
 → Yucatán (Demand: 419, Used: 423)
 → Quintana Roo (Demand: 1286, Used: 1709)
 → Return to Depot (Depot)
Distance: 3298 km


🚚 Route for vehicle 3 (Warehouse: Depot):
 → Depot (Demand: 0, Used: 0)
 → Querétaro (Demand: 132, Used: 132)


# Evaluation

In [42]:
print(f"📦 Baseline distance: {baseline_total_distance}")
print(f"🏪 2-warehouse distance: {total_distance_with_2_warehouses}")

📦 Baseline distance: 15365
🏪 2-warehouse distance: 17992


In [43]:
improvement = total_distance_with_2_warehouses - baseline_total_distance
improvement_pct = improvement / baseline_total_distance * 100

print(f"\n📉 Distance changed by: {improvement:.2f} km ({improvement_pct:.2f}%)")


📉 Distance changed by: 2627.00 km (17.10%)
